[Nvidia Dali ](http://https://docs.nvidia.com/deeplearning/dali/user-guide/docs/index.html) is a GPU based library for very fast data loading and preprocessing.  It contains GPU based image decoders which can be used for fast and parallel decoding of jpeg2000 images.  This notebook contains a minimal example of extracting the jpeg2000 encoded images conatained in a dicom container and decoding them on GPU.

![](https://arcwiki.rs.gsu.edu/nvidia_dali_pipeline.png)

**Note:** Need to use a nightly build of DALI because UINT16 support was recently added and is not in the main prod wheel

In [1]:
!pip install -qU python-gdcm pydicom pylibjpeg
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist/nightly --upgrade nvidia-dali-nightly-cuda110

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist/nightly
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.5/446.5 MB 2.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import pydicom
import glob, os
import pydicom
from pydicom.filebase import DicomBytesIO
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.types import DALIDataType

In [3]:
#here's the magic of hacking the jpeg2000 encoded bitstream.  Function saves jp2 encoded image files contained within dicom which will be later decoded by DALI 
def convert_dicom_to_j2k(file):
    patient = file.split('/')[-2]
    image = file.split('/')[-1][:-4]
    dcmfile = pydicom.dcmread(f'../input/rsna-breast-cancer-detection/train_images/{file}')
    if dcmfile.file_meta.TransferSyntaxUID=='1.2.840.10008.1.2.4.90':
        with open(f'../input/rsna-breast-cancer-detection/train_images/{file}', 'rb') as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(b"\x00\x00\x00\x0C")  #<---- the jpeg2000 header info we're looking for
        hackedbitstream = bytearray()
        hackedbitstream.extend(ds.PixelData[offset:])
        with open(f"../working/{patient}_{image}.jp2", "wb") as binary_file:
            binary_file.write(hackedbitstream)

In [4]:
df = pd.read_csv('/kaggle/input/dali-rsna-helpers/dicom_jpg_formats.csv')
del df['label']
df['uid'].value_counts()

1.2.840.10008.1.2.4.70    29519
1.2.840.10008.1.2.4.90    25187
Name: uid, dtype: int64

1.2.840.10008.1.2.4.70  ----> JPEG Lossless, Nonhierarchical, First- Order Prediction (Processes 14)

1.2.840.10008.1.2.4.90  ----> JPEG 2000 Image Compression (Lossless Only) 

**For now this solution applies only to the JPEG 2000 Image Compression standard**


In [5]:
allj2kdicoms = df[df['uid']=='1.2.840.10008.1.2.4.90']['dicom'].tolist()

Let's get our initial baseline of time to decode a single image using pydicom (CPU based decoding).

In [6]:
def pydicom_benchmark(file):
    dicom = pydicom.dcmread(f'../input/rsna-breast-cancer-detection/train_images/{file}')
    img = dicom.pixel_array

In [7]:
_ = Parallel(n_jobs=2)(delayed(pydicom_benchmark)(uid) for uid in tqdm(allj2kdicoms[:32]))

  0%|          | 0/32 [00:00<?, ?it/s]

Now to the GPU based vesion. 

Step 1: convert dicoms to .jp2 files (jpeg2000 encoded images)

In [8]:
_ = Parallel(n_jobs=2)(delayed(convert_dicom_to_j2k)(uid) for uid in tqdm(allj2kdicoms[:32]))

  0%|          | 0/32 [00:00<?, ?it/s]

Step 2: create [DALI pipeline](https://docs.nvidia.com/deeplearning/dali/user-guide/docs/pipeline.html).  

All we're doing in the pipeline is reading and decoding the j2k files and returining the uint16 array (equivalent to pydicom.pixel_array function)

In [9]:
j2kfiles = [f'../working/{thing.split("/")[-2]}_{thing.split("/")[-1][:-4]}.jp2' for thing in allj2kdicoms[:32]]

In [10]:
@pipeline_def
def j2k_decode_pipeline():
    jpegs, _ = fn.readers.file(files = j2kfiles)
    images = fn.experimental.decoders.image(jpegs, device='mixed', output_type=types.ANY_DATA, dtype=DALIDataType.UINT16)
    return images

In [11]:
max_batch_size = 32
pipe = j2k_decode_pipeline(batch_size=max_batch_size, num_threads=2, device_id=0, debug=True)
pipe.build()

In [12]:
%%time
pipe_out = pipe.run()

CPU times: user 2.89 s, sys: 346 ms, total: 3.23 s
Wall time: 2.03 s


Wow nice speedup, that's the time to decode **32 images** AND the data is sitting on the GPU already.  How much memory are we using?

In [13]:
!nvidia-smi

Sat Dec 10 17:01:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    39W / 250W |   3361MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Let's test and make sure the array we get from pydicom is the same as what DALI gives us after decode

In [14]:
pydicom_image = pydicom.dcmread(f'../input/rsna-breast-cancer-detection/train_images/{allj2kdicoms[30]}').pixel_array

In [15]:
pydicom_image

array([[2652, 2525, 2791, ...,    0,    0,    0],
       [2720, 2841, 2678, ...,    0,    0,    0],
       [2977, 2490, 2829, ...,    0,    0,    0],
       ...,
       [1990, 1809, 1650, ...,    0,    0,    0],
       [1914, 1969, 1909, ...,    0,    0,    0],
       [1788, 1960, 1914, ...,    0,    0,    0]], dtype=uint16)

In [16]:
testfile = [f'../working/{thing.split("/")[-2]}_{thing.split("/")[-1][:-4]}.jp2' for thing in [allj2kdicoms[30]]]


@pipeline_def
def j2k_decode_pipeline():
    jpegs, _ = fn.readers.file(files = testfile)
    images = fn.experimental.decoders.image(jpegs, device='mixed', output_type=types.ANY_DATA, dtype=DALIDataType.UINT16)
    return images

max_batch_size = 1
pipe = j2k_decode_pipeline(batch_size=max_batch_size, num_threads=2, device_id=0, debug=True)
pipe.build()

pipe_out = pipe.run()

dali_image = pipe_out[0].as_cpu().as_array()[0,:,:,0]

In [17]:
dali_image

array([[2652, 2525, 2791, ...,    0,    0,    0],
       [2720, 2841, 2678, ...,    0,    0,    0],
       [2977, 2490, 2829, ...,    0,    0,    0],
       ...,
       [1990, 1809, 1650, ...,    0,    0,    0],
       [1914, 1969, 1909, ...,    0,    0,    0],
       [1788, 1960, 1914, ...,    0,    0,    0]], dtype=uint16)

In [18]:
np.all(dali_image==pydicom_image)

True